# Image Analysis 2021

This Jupyter notebook is part of the course Image Analysis from Radboud University (Nijmegen, Netherlands), and it was developed by researchers of Radboud University Medical Center (Nijmegen, Netherlands).

You should have obtained this notebook by downloading it from the official Brightspace page of the course.
If this is not the case, you should contact the course coordinator at this email address: geert.litjens@radboudumc.nl

This notebook formulates an assignment as part of the course, and the content of this notebook should be used solely to develop a solution to this assignment.
You should not make the code provided in this notebook, or your own solution, publicly available.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` by substituting `None` variables or by adding your own solution and any place that says "YOUR ANSWER HERE" with your answers to the questions. Note that it is perfectly fine to substitute the images in the exercises with your own if you want to. Please fill in your name and collaborators below:

## Students
Please fill in this cell with your name and e-mail address. This information will be used to track completion of the assignments.

* Name student #1, email address: ...
* Name student #2, email address (optional): ...

## Instructions

* Groups: You should work in **groups of maximum 2 people**.
* Deadline for this assignment: 
 * Preferably before April 28th
 * Send your **fully executed** notebook to: geert.litjens@radboudumc.nl
 * Or upload to BrightSpace
* The file name of the notebook you submit must be ```NameSurname1_NameSurname2.ipynb```

This notebooks contains cells with snippets of code that we provide in order to load and visualize data, but also some convenience functions that could be useful to develop your assignment.

We also provide templates for functions that have to be implemented, with a given list of input variables and some output variables.

Your submission should contain the **fully executed** notebook with **your code** implemented, as well as **your answers** to questions.

## Libraries

First, we import the basic libraries necessary to develop this assignment.

In [ ]:
import skimage as ski # For reading images
import skimage.transform as skit # Basic image transformation functions
import skimage.segmentation as ssegm
import skimage.draw as sdraw
from   skimage.filters import threshold_otsu
from   skimage.color import label2rgb
import skimage.morphology as smorp
import matplotlib.pyplot as plt # Plotting and visalization of images
import numpy as np # Basic math and array functions

# Thresholding
We'll start with a grayscale image. Let's see how well regular and automated thresholing techniques compare!

In [ ]:
page = ski.data.page() # Get page image as Numpy array
plt.imshow(page, cmap='gray'); # Show the image
print("Image shape: " + str(page.shape)) # Print the shape (i.e. dimenions) of the image

As a first exercise, we'll try to separte the text from the background. A sensible first attempt would be to try to manually define a threshold that might work. We can just do trial and error, but looking at an image histogram is generally a sensible strategy:

In [ ]:
# Note: the histogram uses one range more (257 vs. 256) because the last bin has a closed end (e.g. with
# 256 the last bin will cover both 254 and 255, which we do not want)
hist, bins = np.histogram(page, range(257))
plt.bar(range(256), hist);

Not a very straightforward histogram to select a threshold for. The background is white, so that's probably the peak at the end. Let's try a threshold:

In [ ]:
t = 200

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 4)) 
axes[0].imshow(page, cmap="gray"); # Show the orignal image
axes[1].imshow(page > t, cmap="gray"); # Show the thresholded image
plt.show()

That is not a great result. We can manually optimize it, but we can also use an automated thresholding methods. Let's try to implement Otsu's method from the lecture. It operates under the assumption that we want to discriminate two classes, which we want to do here.

Remember from the lecture that Otsu's threshold aims to minimize within-class variance, which is given as:
$$\sigma _{w}^{2}(t)=\omega _{0}(t)\sigma _{0}^{2}(t)+\omega _{1}(t)\sigma _{1}^{2}(t)$$
this is the same as maximizing the between class variance (i.e. making the distance in value between the average pixel in class 1 and class 2 as high as possible). This can be written as:
$$\sigma _{b}^{2}(t)=\omega _{0}(t)\omega _{1}(t)[\mu _{0}(t) - \mu _{1}(t)]^{2}$$
Here $\omega _{0}$ and $\omega _{1}$ are the probabilities that a pixel belongs to class 0 and class 1, which can be calculated by counting the number of pixels in each class (which depends on the threshold) and then subdividing by the total number of pixels in the image. Furthermore, $\mu _{0}$ and $\mu _{1}$ are the average pixel values for each class. To find the optimal threshold, you will need to calculate these four values for every possible threshold, calculate $\sigma _{b}^{2}$, and pick the threshold for which this value is highest.

In [ ]:
# Replace the None's in the function below
def otsu_threshold(hist, bins):
    inter_class_variances = []
    total_pixels = np.sum(hist)
    for t in range(1, 255): # Iterate over all possible threshold values (excluding the first and final values)
        
        # Get the weights
        weight1 = None
        weight2 = None

        # Get the class means mu0(t)
        mean1 = None
        # Get the class means mu1(t)
        mean2 = None

        inter_class_variances.append(weight1 * weight2 * (mean1 - mean2) ** 2)

    # Maximize the inter_class_variance function val
    index_of_max_val = np.argmax(inter_class_variances)

    threshold = bins[:-1][index_of_max_val]
    return threshold

We can compare our implementation against the implementation in scikit-image to see if it is correct.

In [ ]:
t_otsu = otsu_threshold(hist, bins)
t_otsu_skimage = threshold_otsu(page, nbins=256)
print("Our implementation: " + str(t_otsu) + ", official implementation: " + str(t_otsu_skimage))

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 4)) # This creates a plot with 3 horizontal subplots
axes[0].imshow(page, cmap="gray");
axes[1].imshow(page > t, cmap="gray");
axes[2].imshow(page > t_otsu, cmap="gray");
axes[3].imshow(page > t_otsu_skimage, cmap="gray");
plt.show()

<font color="blue">**Question:** Even with the automated threshold calculation, we do not get a perfect segmentation result due to the background illumination. Can you come up with a strategy that improves on this result, but while using the exact same methods?

# Connected Component Analysis and Region Growing
Now let's move to instance segmentation, where we try to identify every single entity of a class in an image separately.

### Connected component analysis

First, let's load an image which has a lot of different instances of the same class:

In [ ]:
coins = ski.data.coins()
plt.imshow(coins, cmap='gray');

We will use a slightly more fancy method to segment the individual samples, specifically Sobel filtering (which we will discuss in the next lecture) and a Watershed Transformation (https://www.wikiwand.com/en/Watershed_(image_processing). 

In [ ]:
edges = ski.filters.sobel(coins)
markers = np.zeros_like(coins)
foreground, background = 1, 2
markers[coins < 30.0] = background
markers[coins > 150.0] = foreground

segmented_coins = ssegm.watershed(edges, markers) == 1

This results in a nice semantic segmentation, but all the instances have the same label, as you can see below:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

ax[0].imshow(coins, cmap=plt.cm.gray)
ax[0].set_title("Original")

label_image = label2rgb(segmented_coins, coins, bg_label=0, bg_color=None, kind="overlay")
ax[1].imshow(label_image)
ax[1].set_title("Segmented coins")

fig.tight_layout()
plt.show()

Below I have implemented the two functions needed to perform connected component analysis. The first handles the connectivity of the pixel under consideration and currently assumes 4-connectivity. The latter uses this function to perform the connected component analysis.

In [ ]:
def check_nbs(label_image, row, col, connected_labels):
    # We start with the assumption that the previous label is 0
    prev_label = 0
    
    # For 4-connectivity, we need to check two potential previous labels, left and up from the current position,
    # in 8-connectivity we also need to check the diagonal.
    up_label = label_image[row - 1, col]
    left_label = label_image[row, col - 1]
    
    # First we check whether any of the previous labels is 0, because then we can stop because there is no previous
    # label in the neighborhood
    if up_label > 0 or left_label > 0:
        # Subsequently, we need to check whether one previous label is higher than the other, because we want to keep
        # track of connected labels (to later replace them) and because we want to return the lowest of the two.
        if up_label > left_label:
            # Here we check whether the lower valued label is actually higher than 0, otherwise there is only
            # one previous label and we can simply return that. Otherwise we need to register the connectivity
            # between the different labels and return the lowest label.
            if left_label > 0:
                connected_labels[up_label].add(left_label)
                return left_label
            else:
                return up_label
        # Repetion of the statement above
        elif left_label > up_label:
            if up_label > 0:
                connected_labels[left_label].add(up_label)
                return up_label
            else:
                return left_label
        else:
            return up_label
    else:
        return prev_label

In [ ]:
def get_connected_components(image):
    label_image = np.zeros_like(image, dtype="uint32")
    connected_labels = {}
    cur_label = 0
    # First pass, iterate over all pixels
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            # If we encounter an object (value = 1), check whether one of the earlier visited neighbors also was
            # an object and which label it got. The neighbors that are checked depend on the connectivity (4 or 8 in 2D)
            if image[row, col] == 1:
                prev_label = check_nbs(label_image, row, col, connected_labels)
                if prev_label == 0: # New object found
                    cur_label += 1
                    label_image[row, col] = cur_label
                    connected_labels[cur_label] = set()
                else:
                    label_image[row,col] = prev_label
    
    # Here we remap every label to the lowest label it is connected to
    mapping = [0]
    for k in sorted(connected_labels.keys()):
        if connected_labels[k]:
            cur_lab = k
            while connected_labels[cur_lab]:
                cur_lab = min(connected_labels[cur_lab])
            mapping.append(cur_lab)
        else:
            mapping.append(k)
        
    # Second pass relabels all labels based on the new mapping
    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            label_image[row, col] = mapping[label_image[row, col]]
    return label_image

Below shows the result of the connected component analysis: all instances are nicely separated.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

ax[0].imshow(coins, cmap=plt.cm.gray)
ax[0].set_title("Original")

labeled_coins = get_connected_components(segmented_coins)
label_image = label2rgb(labeled_coins, coins, bg_label=0, bg_color=None, kind="overlay")
ax[1].imshow(label_image)
ax[1].set_title("Segmented coins")

fig.tight_layout()
plt.show()

**Assignment**: Below you have a test image for which we can also identify the connected components. However, 4-connectedness is not enoug here. Can you modify the `check_nbs` function above to handle 8-connectivity? You do not need to modify the other function.

In [ ]:
test_image = np.zeros((220, 220), dtype="uint8")
test_image[10:50, 10:50] = 1
test_image[50:100, 50:100] = 1
test_image[10:50, 100:140] = 1
test_image[150:200, 150:200] = 1

In [ ]:
plt.imshow(get_connected_components(test_image));

### Region growing

We performed the instance segmentation with a relatively complicated set of methods. We can also use an easier method, based on region growing, that requires only a single algorithm. However, it does require some manual work.

In [ ]:
coins = ski.data.coins()
plt.imshow(coins, cmap='gray');

Below we segment the first coin using the `flood` function. We need to specify a tolerance which determines how similar the gray value pf the adjacent pixels has to be to the seed point to be included in the segmentation. You can visualize this as well below.

In [ ]:
seed_point = (50, 50)
tolerance = 30
coin_mask = ssegm.flood(coins, seed_point, connectivity=2, tolerance=tolerance).astype("ubyte");

**Assignment:** This cell extracts the rest of the coins from the first row. Modify seed_points and tolerances to segment all coins.

In [ ]:
seed_points = [None]
tolerances = [None]
for coin_nr in range(5):
    coin_mask += (coin_nr + 2) * ssegm.flood(coins, seed_points[coin_nr], connectivity=2, tolerance=tolerances[coin_nr]).astype("ubyte");

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

ax[0].imshow(coins, cmap=plt.cm.gray)
ax[0].scatter(seed_point[1], seed_point[0])
ax[0].set_title("Original")

label_image = label2rgb(coin_mask, coins, bg_label=0, bg_color=None, kind="overlay")
ax[1].imshow(label_image)
ax[1].scatter(seed_point[1], seed_point[0])
ax[1].set_title("Segmented coin")

fig.tight_layout()
plt.show()

# Mathematical Morphology

The last topic of this week is mathematical morphology. We will experiment with different use-cases, both using standard morphological operations, reconstruction, and greyscale morphology.

### Erosion and dilation

Below is the first binary image we will experiment with:

In [ ]:
horse = ski.data.horse().astype("ubyte")
plt.imshow(horse, cmap="gray"); # Show the image
print("Image shape: " + str(horse.shape)) # Print the shape (i.e. dimenions) of the image

**Assignment:** Extract the border from the horse using a morphological operation. You can either use `smorp.erosion` or `smorp.dilation`. You need to do one extra step in addition to the morphological operation. Remember you can use `?smorp.erosion` to get an explanation of the function.

In [ ]:
horse_border = None

Show the results below:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 8))

ax[0].imshow(horse, cmap=plt.cm.gray)
ax[0].set_title("Original")
ax[1].imshow(horse_border, cmap=plt.cm.gray)
ax[1].set_title("Border")
fig.tight_layout()
plt.show()

### Hole filling using morphological reconstruction

The function below creates and artifical image with holes. Let's try to get them filled!

In [ ]:
holes_image = np.zeros((256, 256), dtype="ubyte")
holes_image[50:100, 50:100] = 1
holes_image[55:65, 55:65] = 0
holes_image[85:90, 75:80] = 0
holes_image[175:225, 175:225] = 1
holes_image[200:215, 200:215] = 0
holes_image[105:115, 50:100] = 1
circ_pos = sdraw.disk((75, 200), 30)
circ_hole_pos = sdraw.disk((75, 200), 10)
holes_image[circ_pos] = 1
holes_image[circ_hole_pos] = 0

In [ ]:
plt.imshow(holes_image, cmap="gray");

First, let's try to use a standard *closing* operation, which is a dilation followed by an erosion. We will use a 20-pixel disk as a structuring element. The plot function shows tbe dilated and subsequently the eroded image.

In [ ]:
dilated_image = smorp.dilation(holes_image, selem=smorp.disk(20))
eroded_image =  smorp.erosion(dilated_image, selem=smorp.disk(20))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))

ax[0].imshow(dilated_image, cmap=plt.cm.gray)
ax[0].set_title("Original")
ax[1].imshow(eroded_image, cmap=plt.cm.gray)
ax[1].set_title("Contrast Strecthced")
fig.tight_layout()
plt.show()

As you can see, the holes are filled, but we have an artificat: the bar below the top-left square is now fused with the square, which is not what we want. Let's see if we can do better with morphological reconstruction.

Remember that for morphological reconstruction we work with two images: a marker image on which the morphological operations are applied and the mask image, which is used to intersect and limit the marker image. Both are created in the cell below. The marker image is an image filled with 0's, except for the borders which are filled with 1's and are the starting points for our morphological operation.

In [ ]:
marker_image = np.ones_like(holes_image)
marker_image[1:-1,1:-1] = 0
mask_image = 1 - holes_image

**Assignment:** Adapt the function below to perform the reconstruction. You will need to replace the `None` statements. The first needs to apply the morphological operation, the second needs to use the mask image.

In [ ]:
recon_image = marker_image.copy()
last_image = np.zeros_like(marker_image)
while (recon_image != last_image).any():
    last_image = recon_image.copy()
    dilated_image = None
    recon_image = None

The function below should show the test image with the holes filled, but without any of the artifacts!

In [ ]:
plt.imshow(1 - recon_image, cmap = "gray");

### Greyscale Morphology

The last topic is greyscale morphological reconstruction. Specifically, we are going to use it to correct illumincation differences and improve our thresholding operations. You do not need to implement anything, but you have to explain what is happening in your own words. First we make the marker and mask images:

In [ ]:
marker_image = np.copy(ski.data.page())
marker_image[1:-1, 1:-1] = ski.data.page().max()
mask_image = ski.data.page()

Now perform the reconstruction:

In [ ]:
recon_image = marker_image.copy()
last_image = np.zeros_like(marker_image)
while (recon_image != last_image).any():
    last_image = recon_image.copy()
    eroded_image = smorp.erosion(recon_image, selem=np.ones((3,3)))
    recon_image = np.maximum(eroded_image, mask_image)


Calculate the illumination corrected image:

In [ ]:
corrected = recon_image - ski.data.page()

In [ ]:
plt.imshow(corrected, cmap="gray");

In [ ]:
t_otsu_improved = threshold_otsu(corrected, nbins=256)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4)) # This creates a plot with 3 horizontal subplots
axes[0].imshow(page, cmap="gray"); 
axes[1].imshow(page > t_otsu, cmap="gray");
axes[2].imshow(corrected < t_otsu_improved, cmap="gray");
plt.show()

<font color="blue">**Question:** Explain, based on the definition of greyscale morphological operations, what the above process is doing in each step. For example, what happens when we perform the greyscale erosion on the marker image? You can plot the intermediate images if you want to see it visually.